In [2]:
import pandas as pd
import random
import os 
import sys

from importlib import reload

sys.path.append(os.path.abspath('..'))

In [3]:
# Change Tracking vs Change Data Capture

In [4]:
import SharedModule
reload(SharedModule)
from SharedModule import CSV

In [5]:
cursor = CSV('../MyDatabase')

In [6]:
def ProcessCDC(previous_snapshot, new_snapshot):
    '''
    previous_snapshot: pd.DataFrame
    new_snapshot: pd.DataFrame

    '''
    ...

In [7]:
number_of_customers = 5

previous= pd.DataFrame(data = {
    'CUSTOMER_ID':[i+1 for i in range(0,number_of_customers)],
    })

random.seed(123)

previous['CUSTOMER_TYPE'] = previous.apply(
    lambda x: 'New' if random.randint(0,100) > 15 else 'Existing',axis = 1)

previous['LOAN_STATUS'] = previous.apply(
    lambda x: 'Current' if random.randint(0,100) > 30 else 'Delinquent',axis = 1)

previous['LOAN_DATE'] = pd.to_datetime('2024-01-01')

#previous_snapshot['DATE_ENTERED'] = pd.to_datetime('2024-01-01')

previous['LAST_UPDATE'] = pd.to_datetime('2024-01-14')

previous

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,LAST_UPDATE
0,1,Existing,Current,2024-01-01,2024-01-14
1,2,New,Delinquent,2024-01-01,2024-01-14
2,3,Existing,Delinquent,2024-01-01,2024-01-14
3,4,New,Current,2024-01-01,2024-01-14
4,5,New,Current,2024-01-01,2024-01-14


In [8]:
cursor.SET('Previous',previous)

In [9]:
new_snapshot =pd.concat([pd.DataFrame(data={
    'CUSTOMER_ID':[1,3,6,7],
    'CUSTOMER_TYPE':['Existing','Existing','New','New'],
    'LOAN_STATUS':['Delinquent','Current','Not Due','Not Due'],
    'LOAN_DATE':[pd.to_datetime('2024-01-01'),pd.to_datetime('2024-01-01'),pd.to_datetime('2024-01-15'),pd.to_datetime('2024-01-15')],
    #'DATE_ENTERED':[pd.to_datetime('2024-01-07'),pd.to_datetime('2024-01-07'),pd.to_datetime('2024-01-16'),pd.to_datetime('2024-01-16')],
    }),
    previous.query('CUSTOMER_ID in [2,4,5]')])

#new_snapshot['DATE_ENTERED'] = pd.to_datetime('2024-01-16')
del new_snapshot['LAST_UPDATE']
new_snapshot.sort_values(by = 'CUSTOMER_ID')

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE
0,1,Existing,Delinquent,2024-01-01
1,2,New,Delinquent,2024-01-01
1,3,Existing,Current,2024-01-01
3,4,New,Current,2024-01-01
4,5,New,Current,2024-01-01
2,6,New,Not Due,2024-01-15
3,7,New,Not Due,2024-01-15


In [10]:
cursor.SET('New_Snapshot',new_snapshot)

In [11]:
total_snapshot = pd.concat([previous,new_snapshot])

total_snapshot.sort_values(by = 'CUSTOMER_ID').reset_index(drop = True)
# Find Rows that are new and not duplicates by combining the old and new snapshot;

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,LAST_UPDATE
0,1,Existing,Current,2024-01-01,2024-01-14
1,1,Existing,Delinquent,2024-01-01,NaT
2,2,New,Delinquent,2024-01-01,2024-01-14
3,2,New,Delinquent,2024-01-01,NaT
4,3,Existing,Delinquent,2024-01-01,2024-01-14
5,3,Existing,Current,2024-01-01,NaT
6,4,New,Current,2024-01-01,2024-01-14
7,4,New,Current,2024-01-01,NaT
8,5,New,Current,2024-01-01,2024-01-14
9,5,New,Current,2024-01-01,NaT


In [12]:
 # if columns (excluding last_update) are the same then we can keep the record from previous snapshot.
deduped = total_snapshot.sort_values(by = ['CUSTOMER_ID','LAST_UPDATE']).drop_duplicates(subset = [c for c in total_snapshot.columns if c != 'LAST_UPDATE']).reset_index(drop = True)
deduped

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,LAST_UPDATE
0,1,Existing,Current,2024-01-01,2024-01-14
1,1,Existing,Delinquent,2024-01-01,NaT
2,2,New,Delinquent,2024-01-01,2024-01-14
3,3,Existing,Delinquent,2024-01-01,2024-01-14
4,3,Existing,Current,2024-01-01,NaT
5,4,New,Current,2024-01-01,2024-01-14
6,5,New,Current,2024-01-01,2024-01-14
7,6,New,Not Due,2024-01-15,NaT
8,7,New,Not Due,2024-01-15,NaT


In [13]:
# The rows where last_update are Null are the ones that are new to previous snapshot; lets hold them
new_rows = deduped[deduped.LAST_UPDATE.isna()].copy()
new_rows

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,LAST_UPDATE
1,1,Existing,Delinquent,2024-01-01,NaT
4,3,Existing,Current,2024-01-01,NaT
7,6,New,Not Due,2024-01-15,NaT
8,7,New,Not Due,2024-01-15,NaT


In [14]:
# We can add the date we found these:
new_rows.LAST_UPDATE = new_rows.LAST_UPDATE.fillna(pd.to_datetime('2024-01-15'))
new_rows

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,LAST_UPDATE
1,1,Existing,Delinquent,2024-01-01,2024-01-15
4,3,Existing,Current,2024-01-01,2024-01-15
7,6,New,Not Due,2024-01-15,2024-01-15
8,7,New,Not Due,2024-01-15,2024-01-15


In [15]:
new_previous = pd.concat([previous,new_rows]).sort_values(by = ['CUSTOMER_ID','LAST_UPDATE']).reset_index(drop = True)
new_previous

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,LAST_UPDATE
0,1,Existing,Current,2024-01-01,2024-01-14
1,1,Existing,Delinquent,2024-01-01,2024-01-15
2,2,New,Delinquent,2024-01-01,2024-01-14
3,3,Existing,Delinquent,2024-01-01,2024-01-14
4,3,Existing,Current,2024-01-01,2024-01-15
5,4,New,Current,2024-01-01,2024-01-14
6,5,New,Current,2024-01-01,2024-01-14
7,6,New,Not Due,2024-01-15,2024-01-15
8,7,New,Not Due,2024-01-15,2024-01-15


In [16]:
new_previous_snapshot = new_previous.drop_duplicates(subset = 'CUSTOMER_ID',keep = 'last').reset_index(drop = True)
new_previous_snapshot

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,LAST_UPDATE
0,1,Existing,Delinquent,2024-01-01,2024-01-15
1,2,New,Delinquent,2024-01-01,2024-01-14
2,3,Existing,Current,2024-01-01,2024-01-15
3,4,New,Current,2024-01-01,2024-01-14
4,5,New,Current,2024-01-01,2024-01-14
5,6,New,Not Due,2024-01-15,2024-01-15
6,7,New,Not Due,2024-01-15,2024-01-15


In [18]:

def get_snapshot(previous,primary_key = 'CUSTOMER_ID',update_col = 'LAST_UPDATE',):
    """
    previous: pd.DataFrame
        DataFrame with duplciated rows. 
    """
    return previous.sort_values(by=[primary_key,update_col]).drop_duplicates(subset = primary_key,keep = 'last').reset_index(drop = True)

def find_new_rows(previous_snapshot,new_snapshot,update_col = 'LAST_UPDATE',ignore_col=[]):
    '''
    previous_snapshot: pd.DataFrame
        DataFrame with no duplicate rows. The DataFrame from Yesterday.
    new_snapshot: pd.DataFrame
        DataFrame with no duplicate rows. The DataFrame from Today.
    
    return: pd.DataFrame
        DataFrame with no duplicate rows. The differences between Today and Yesterday.
    '''
    compare_col = [c for c in previous_snapshot.columns if c not in ignore_col + [update_col]]

    previous_snapshot[compare_col]
    new_snapshot[compare_col]

    temp = pd.concat([previous_snapshot,new_snapshot[compare_col]])
    temp_deduped = temp.drop_duplicates(subset = compare_col,keep = False).reset_index(drop = True)

    # Remove rows that were from yesterday
    temp_deduped = temp_deduped[temp_deduped.LAST_UPDATE.isna()]
    
    return temp_deduped

def process_CDC(previous,new_snapshot,refresh_date = pd.to_datetime('2024-01-15'), primary_key = 'CUSTOMER_ID',update_col = 'LAST_UPDATE',ignore_col=[]):

    previous_snapshot = get_snapshot(previous,primary_key,update_col)
    new_rows = find_new_rows(previous_snapshot,new_snapshot,update_col,ignore_col)
    new_rows[update_col] = refresh_date
    
    return pd.concat([previous,new_rows])

print('Previous Snapshot:')

print(get_snapshot(previous))
print()

print('Rows new to today:')
print(find_new_rows(get_snapshot(previous),new_snapshot).sort_values(by='CUSTOMER_ID'))
print()

print('New Historical Data')
print(pd.concat([previous,find_new_rows(get_snapshot(previous),new_snapshot).fillna(pd.to_datetime('2024-01-15'))]).sort_values(by=['CUSTOMER_ID','LAST_UPDATE']))
print()

print('New Previous Snapshot:')
print(get_snapshot(pd.concat([previous,find_new_rows(get_snapshot(previous),new_snapshot).fillna(pd.to_datetime('2024-01-15'))])).sort_values(by=['CUSTOMER_ID']))


Previous Snapshot:
   CUSTOMER_ID CUSTOMER_TYPE LOAN_STATUS  LOAN_DATE LAST_UPDATE
0            1      Existing     Current 2024-01-01  2024-01-14
1            2           New  Delinquent 2024-01-01  2024-01-14
2            3      Existing  Delinquent 2024-01-01  2024-01-14
3            4           New     Current 2024-01-01  2024-01-14
4            5           New     Current 2024-01-01  2024-01-14

Rows new to today:
   CUSTOMER_ID CUSTOMER_TYPE LOAN_STATUS  LOAN_DATE LAST_UPDATE
2            1      Existing  Delinquent 2024-01-01         NaT
3            3      Existing     Current 2024-01-01         NaT
4            6           New     Not Due 2024-01-15         NaT
5            7           New     Not Due 2024-01-15         NaT

New Historical Data
   CUSTOMER_ID CUSTOMER_TYPE LOAN_STATUS  LOAN_DATE LAST_UPDATE
0            1      Existing     Current 2024-01-01  2024-01-14
2            1      Existing  Delinquent 2024-01-01  2024-01-15
1            2           New  Delinquent 202

In [20]:
process_CDC(previous,new_snapshot).sort_values(by = ['CUSTOMER_ID','LAST_UPDATE'])

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,LAST_UPDATE
0,1,Existing,Current,2024-01-01,2024-01-14
2,1,Existing,Delinquent,2024-01-01,2024-01-15
1,2,New,Delinquent,2024-01-01,2024-01-14
2,3,Existing,Delinquent,2024-01-01,2024-01-14
3,3,Existing,Current,2024-01-01,2024-01-15
3,4,New,Current,2024-01-01,2024-01-14
4,5,New,Current,2024-01-01,2024-01-14
4,6,New,Not Due,2024-01-15,2024-01-15
5,7,New,Not Due,2024-01-15,2024-01-15


In [ ]:
def ProcessCDC(previous_snapshot, new_snapshot, refresh_date = pd.to_datetime('2024-01-15')):
    '''
    previous_snapshot: pd.DataFrame
    new_snapshot: pd.DataFrame

    # Date_Entered will always be new; how do u track?

    # If Date_Entered is the only difference then drop it, otherwise keep?

    return:
    new previous_snapshot :) : pd.DataFrame
    '''
    new_snapshot['LAST_UPDATE'] = None

    temp = pd.concat([previous_snapshot,new_snapshot])
    temp = temp.drop_duplicates(subset = [c for c in previous_snapshot.columns if c != 'LAST_UPDATE'])

    new = temp[temp.LAST_UPDATE == None].copy()

    new.LAST_UPDATE = new.LAST_UPDATE.fillna(refresh_date)

    #temp.LAST_UPDATE = temp.LAST_UPDATE.fillna(refresh_date)

    return new


In [ ]:
new_previous_snapshot = ProcessCDC(previous_snapshot,new_snapshot).sort_values(by = ['CUSTOMER_ID','LAST_UPDATE']).reset_index(drop = True)

In [ ]:
new_previous_snapshot

In [ ]:
current = new_previous_snapshot.sort_values(by = ['CUSTOMER_ID','LAST_UPDATE']).drop_duplicates(subset = 'CUSTOMER_ID',keep = 'last').reset_index(drop = True)
current

In [ ]:
from datetime import timedelta
t = new_previous_snapshot.copy()
t_c = current.copy()
t_c.LAST_UPDATE = None
example_date = pd.to_datetime('2024-01-16')
for i in range(0,35):
    example_date = example_date + timedelta(days=1)
    t_c['DATE_ENTERED'] = example_date
    if i == 14:
        t_c.loc[t_c.CUSTOMER_ID == 1,'LOAN_STATUS'] = 'DPD15'
        #example_date = pd.to_datetime('2024-02-05')
    if i == 30:
        t_c.loc[t_c.CUSTOMER_ID == 1,'LOAN_STATUS'] = 'DPD30'
        #example_date = pd.to_datetime('2024-02-20')
    if i == 33:
        t_c.loc[t_c.CUSTOMER_ID == 1,'LOAN_STATUS'] = 'Current'
        #example_date = pd.to_datetime('2024-04-01')
    t = ProcessCDC(t,t_c,example_date)
    t_c = t.sort_values(by = ['CUSTOMER_ID','LAST_UPDATE']).drop_duplicates(subset = 'CUSTOMER_ID',keep = 'last').reset_index(drop = True).copy()
    
t.sort_values(by = ['CUSTOMER_ID','LAST_UPDATE']).drop_duplicates(subset = 'CUSTOMER_ID',keep = 'last').reset_index(drop = True)

In [ ]:
t

In [ ]:
t_c

In [ ]:
t.sort_values(by = ['CUSTOMER_ID','LAST_UPDATE'])
# Why even keep old values?

In [ ]:
new_previous_snapshot